# This notebook aims to reproduce Table 1


## Install the following packages if necessary


In [1]:
#install.packages('xml2')
#install.packages('neat')
#install.packages('neat')

#if (!requireNamespace("BiocManager", quietly = TRUE))
#install.packages("BiocManager")

#BiocManager::install("biomaRt")

## Loading the packages


In [2]:
###MainClusterWhitfild
library('igraph')
library('biomaRt')
library('neat')
#library('Oscope')
#load('~/Manchester/Alexis/Oscope++/OscopeRcode/casestudy/WhitfieldStandardOscope/StandardOscope.RData')

#mypath='/home/luisa/Manchester/Alexis/Oscope++/'
customlib='../R/CustomLibrary.R'
source(customlib)

workingdir='../casestudy/Whitfield/Results'





Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

Warning message:
“package ‘biomaRt’ was built under R version 3.6.1”

## loading the data and Graph creation


In [3]:
filename='../casestudy/Whitfield/filt0.9Data_N2000_g50_TFTrue_SummaryPartition.csv'
ppp=strsplit('Whitfield/filt0.9Data_N2000_g50_TFTrue_SummaryPartition.csv',split=".csv")
basefilea=ppp[[1]]
#filename='filt0.9Data_N1000_g18_TFTrue_SummaryPartition.csv'
print(filename)
data=read.csv(file=filename, header= TRUE,row.names=1)
G=graph_from_data_frame(data,directed=FALSE)
#ww=get.edge.attribute(G,'cost')


[1] "../casestudy/Whitfield/filt0.9Data_N2000_g50_TFTrue_SummaryPartition.csv"


In [4]:
dir('../casestudy/Whitfield/')


[1] "CellCycleGeneList_1134.txt"                       
[2] "filt0.9Data_N2000_g50_TFTrue_SummaryPartition.csv"
[3] "ImageIdConversions.txt"                           
[4] "Results"                                          
[5] "StandardOscope.RData"

## Walktrap Community Extraction


In [5]:
###############WALKTRAP COMMUNITY 
wt=walktrap.community(G)
modularity(wt)
mwt=membership(wt)
sz=sizes(wt)
#sz[which(sz>=10)]
Mwt=as.matrix(membership(wt))

[1] 0.2262231

##  Retrieving information and merge communities with Cell Cycle related genes



In [6]:
genes=read.delim(file.path('../casestudy/Whitfield/ImageIdConversions.txt'),sep="\t",header=TRUE,quote="")
details=genes[,c(1:3)]
colnames(details)=c('CloneID','Ref','GeneSymbol')
colnames(Mwt)='CommunityID'

merMwt=merge(details,Mwt,by.x='CloneID',by.y=0,all.y=TRUE)
write.table(merMwt, file=file.path(workingdir,'witfildCommG50.txt'),sep='\t',quote=FALSE,row.names=FALSE)
CC=read.delim('../casestudy/Whitfield/CellCycleGeneList_1134.txt')
#CCmerMwt=merge(CC,merMwt,by.x='CLONEID',by.y='CloneID')
CCmerMwt=merge(CC,Mwt,by.x='CLONEID',by.y=0)

write.table(CCmerMwt, file=file.path(workingdir,'CCwitfildCommG50.txt'), sep='\t',quote=FALSE,row.names=FALSE)



## Testing for communities CC enriched


In [7]:
vecComCC=unique(CCmerMwt$CommunityID)#community arricchite per CC
sizeCCcom=vecComCC
EnrichmentTest=vecComCC
GlobalRatio=vecComCC
RelativeRatio=vecComCC

white=length(unique(CCmerMwt$CLONEID))#TotalCommonTargets 360 CC presenti nella rete
black=length(unique(merMwt$CloneID))-white #Total Non CC, tutti gli altri
validation1=NULL
validation2=NULL

for (h in c(1:length(vecComCC))){
  
  ####neat testing for edges
  genlist1=CCmerMwt$CLONEID[which(CCmerMwt$CommunityID==vecComCC[h])]#CC genes in  cluster vecComCC[h]
  #myList[[length(myList)+1]] <- list(genlist1)
  genlist2=merMwt$CloneID[which(merMwt$CommunityID==vecComCC[h])]#All the genes in Cluster vecComCC[h]
  alist = list('set 1' = genlist1)
  names(alist)=vecComCC[h]
  blist= list('set 2'=genlist2)
  test1 = neat(alist = alist, blist = alist, network = G,
               nettype = 'undirected', nodes = V(G)$name, alpha = 0.1)
  test2 = neat(alist = alist, blist = blist, network = G,
               nettype = 'undirected', nodes = V(G)$name, alpha = 0.1)
  validation1=rbind(validation1,print(test1))
  validation2=rbind(validation2,print(test2))
  
  ####hypergeometric testing for vertex
  sizeCCcom[h]=length(which(CCmerMwt$CommunityID==vecComCC[h]))
  x=sizeCCcom[h]#intersection
  k=length(which(merMwt$CommunityID==vecComCC[h]))#commsize
  EnrichmentTest[h]=1-phyper(x,white,black,k)
  GlobalRatio[h]=x/white
  RelativeRatio[h]=x/k
}
BHE=p.adjust(EnrichmentTest,method='BH')
outCC=cbind(vecComCC,sizeCCcom,GlobalRatio,RelativeRatio,EnrichmentTest,BHE)
colnames(outCC)=c('CC enriched Comm ID', 'size CC Overlapping','Global CC Ratio','Relative CC Ratio','CC Overlapping Significance','CC sig BH corr')
CCvalidation=validation1
outCC=cbind(outCC,CCvalidation)



## Wilcoxon Test on Community Significance


In [8]:
##########################################
IdGE10=which(sz>1)
n10=length(IdGE10)
MatSummary=matrix('NA',n10,6)
colnames(MatSummary)=c("Min.","1st Qu.","Median","Mean","3rd Qu.","Max." )
NumberOfEdges=matrix('NA',n10,1)
CommDensity=matrix('NA',n10,1)
RelativeCommDensity=matrix('NA',n10,1)
PsigCom=matrix('NA',n10,1)
TotalEdges=ecount(G)
for (i in c(1:n10)){
  actCom=IdGE10[i]
  actNodes=which(Mwt==actCom)
  actNumVertex=length(actNodes)
  subG=induced_subgraph(G,actNodes)
  actEdge=E(subG)
  actNumEdges=ecount(subG)
  NumberOfEdges[i]=actNumEdges
  CommDensity[i]=actNumEdges/TotalEdges
  RelativeCommDensity[i]=2*actNumEdges/(actNumVertex^2-actNumVertex)
  
  actCost=get.edge.attribute(subG,'cost',index=actEdge)
  vsumAct=as.vector(summary(actCost))
  MatSummary[i,]=vsumAct
  res=community.significance.test(G, actNodes)
  PsigCom[i]=res$p.value
}
sizes10=sz[IdGE10]
sizes10=data.frame(Community=names(sizes10),NumberOfNodes= as.vector(sizes10))

Report10=data.frame(sizes10,Significance=PsigCom,Density=CommDensity,RelativeDensity=RelativeCommDensity,MatSummary)
Report10sig=Report10[which(as.numeric(as.vector(Report10$Significance))<0.01),]
###########
###basicInfo contains the basic info for the Significant communities:
#ID, Significance, Number of nodes, Density and relative density
#we have 8 communities (25  38  61  70  74  92  124 139) significant
basicInfo=Report10sig[,c(1:5)]#data.frame(sizes10,NumberOfNodes=NumberOfNodes,NumberOfEdges=NumberOfEdges,Significance=PsigCom)
valCC=merge(outCC,basicInfo,by.y='Community',by.x=1)
AAA=merge(basicInfo,outCC,by.x='Community',by.y=1,all.x=TRUE)

Table1=data.frame(Size=as.vector(AAA$NumberOfNodes),CC=as.vector(AAA$'size CC Overlapping'),Density=as.vector(AAA$Density),Realative_density=as.vector(AAA$RelativeDensity),Neat=as.vector(AAA$adjusted_p),Hypergeometic=as.vector(AAA$'CC sig BH corr'), Significance=AAA$Significance)

Table1=Table1[order(Table1$Realative_density,decreasing = TRUE),]

In [9]:
Table1

,Size,CC,Density,Realative_density,Neat,Hypergeometic,Significance
,<int>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>
5,27,NA,0.000840752797119883,0.444444444444444,NA,NA,2.4451146538946e-06
1,265,239,0.0774893828012159,0.411034877072613,0.0000000,0.000000,3.3217035022318e-61
4,1157,99,0.801059564102011,0.222259273326495,0.2823336,0.999917,5.1989280098676e-287
2,131,NA,0.00757216461508612,0.165002935995302,NA,NA,9.4263467841257e-14
3,82,1,0.00198870373164895,0.111111111111111,0.9998680,0.999917,0.000402051421160656


In [10]:
write.table(Table1, file=paste(workingdir,'Table1','.txt',sep=''),sep='\t',quote=FALSE,row.names=FALSE,col.names=TRUE)
